In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm, t, probplot
import matplotlib.pyplot as plt

## Portfolio

- SPY: €300 000
- AEX: €300 000
- RDSA: €10 000
- HSI: €90 000
- Bonds: €300 000

Total: €1 000 000

In [9]:
data = pd.read_csv('dataset.csv', index_col=0)
data.index = pd.to_datetime(data.index)

In [21]:
z = np.log(data[['SPY', 'AEX', 'RDSA.AS', 'HSI', 'USDEUR', 'HKDEUR']])

z[['USDEUR']]=z[['USDEUR']]*1.10
z[['HKDEUR']]=z[['HKDEUR']]*1.20

z['YIELDS'] = data['YIELDS'] / 100
x = z.diff()

c = np.zeros(x.shape)
c[:,-1] = z['YIELDS'] / 250 * 0.3

b = np.zeros(x.shape)
b[:,:] = [0.3, 0.3, 0.01, 0.09, 0.3, 0.09, 0.3]
b[:,-1] *= -np.arange(12, 0, -1/(250))[:len(x)]

l = -(c + b * x)

l

,SPY,AEX,RDSA.AS,HSI,USDEUR,HKDEUR,YIELDS
Date,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-04,-0.000000,-0.000000,-0.000000,-0.000000,0.000780,-0.000133,-0.000045
2010-01-05,-0.000793,0.000885,-0.000067,-0.001862,-0.001330,-0.000446,-0.000955
2010-01-06,-0.000211,0.000474,0.000072,-0.000552,0.000873,0.000253,0.000741
2010-01-07,-0.001264,0.000818,0.000030,0.000593,-0.001976,-0.000601,-0.001325
...,...,...,...,...,...,...,...
2020-04-20,0.001788,-0.001357,0.000027,0.000062,-0.000065,-0.000029,0.000005
2020-04-21,0.009250,0.010115,0.000488,0.002007,-0.000254,-0.000075,0.000001
2020-04-22,-0.006586,-0.008272,-0.000603,-0.000377,-0.001426,-0.000480,0.000009


## Assume normal distribution

In [22]:
mean = -c.mean() - b.mean(axis=0).dot(x.mean(axis=0)) * 1000000
std = np.sqrt(b.mean(0).dot(x.cov()).dot(b.mean(0))) * 1000000
# mean = l.mean().sum() * 1000000
# std = np.sqrt(l.cov().sum().sum()) * 1000000

In [23]:
"VaR 97.5%: €{0:.2f}".format(mean + std * norm.ppf(0.975))

'VaR 97.5%: €11768.53'

In [24]:
"VaR 99%: €{0:.2f}".format(mean + std * norm.ppf(0.99))

'VaR 99%: €14005.48'

In [25]:
"ES 97.5%: €{0:.2f}".format(mean + std * norm.pdf(norm.ppf(0.975)) / (1 - 0.975))

'ES 97.5%: €14075.42'

In [26]:
"ES 99%: €{0:.2f}".format(mean + std * norm.pdf(norm.ppf(0.99)) / (1 - 0.99))

'ES 99%: €16074.43'

## Assume students-t distribution

In [27]:
for v in [3, 4, 5, 6]:
    t_std = std / np.sqrt(v / (v - 2))
    print('---Degrees of freedom = {}---'.format(v))
    print("VaR 97.5%: €{0:.2f}".format(mean + std * t.ppf(0.975, v)))
    print("VaR 99%: €{0:.2f}".format(mean + std * t.ppf(0.99, v)))
    print("ES 97.5%: €{0:.2f}".format(mean + std * t.pdf(t.ppf(0.975, v), v) / (1 - 0.975) * (v + t.ppf(0.975, v)**2) / (v-1)))
          
    print("ES 99%: €{0:.2f}".format(mean + std * t.pdf(t.ppf(0.99, v), v) / (1 - 0.99) * (v + t.ppf(0.99, v)**2) / (v-1)))

---Degrees of freedom = 3---
VaR 97.5%: €19232.39
VaR 99%: €27525.22
ES 97.5%: €30571.13
ES 99%: €42559.27
---Degrees of freedom = 4---
VaR 97.5%: €16753.55
VaR 99%: €22678.95
ES 97.5%: €24184.62
ES 99%: €31676.23
---Degrees of freedom = 5---
VaR 97.5%: €15496.65
VaR 99%: €20346.54
ES 97.5%: €21302.95
ES 99%: €26986.27
---Degrees of freedom = 6---
VaR 97.5%: €14741.58
VaR 99%: €18989.52
ES 97.5%: €19682.39
ES 99%: €24422.56
